In [1]:
import numpy as np
import csv

# Import danych


In [2]:
train_file_path = "../data/iris_trn.txt"
test_file_path = "../data/iris_tst.txt"

train_data = np.genfromtxt(train_file_path, delimiter="  ", skip_header=1, dtype=np.float128)
test_data = np.genfromtxt(test_file_path, delimiter="  ", skip_header=1, dtype=np.float128)
train_data

array([[1. , 5.1, 3.5, 1.4, 0.2],
       [1. , 4.9, 3. , 1.4, 0.2],
       [1. , 4.7, 3.2, 1.3, 0.2],
       [1. , 4.6, 3.1, 1.5, 0.2],
       [1. , 5. , 3.6, 1.4, 0.2],
       [1. , 5.4, 3.9, 1.7, 0.4],
       [1. , 4.6, 3.4, 1.4, 0.3],
       [1. , 5. , 3.4, 1.5, 0.2],
       [1. , 4.4, 2.9, 1.4, 0.2],
       [1. , 4.9, 3.1, 1.5, 0.1],
       [1. , 5.4, 3.7, 1.5, 0.2],
       [1. , 4.8, 3.4, 1.6, 0.2],
       [1. , 4.8, 3. , 1.4, 0.1],
       [1. , 4.3, 3. , 1.1, 0.1],
       [1. , 5.8, 4. , 1.2, 0.2],
       [1. , 5.7, 4.4, 1.5, 0.4],
       [1. , 5.4, 3.9, 1.3, 0.4],
       [1. , 5.1, 3.5, 1.4, 0.3],
       [1. , 5.7, 3.8, 1.7, 0.3],
       [1. , 5.1, 3.8, 1.5, 0.3],
       [1. , 5.4, 3.4, 1.7, 0.2],
       [1. , 5.1, 3.7, 1.5, 0.4],
       [1. , 4.6, 3.6, 1. , 0.2],
       [1. , 5.1, 3.3, 1.7, 0.5],
       [1. , 4.8, 3.4, 1.9, 0.2],
       [2. , 7. , 3.2, 4.7, 1.4],
       [2. , 6.4, 3.2, 4.5, 1.5],
       [2. , 6.9, 3.1, 4.9, 1.5],
       [2. , 5.5, 2.3, 4. , 1.3],
       [2. , 6

In [3]:
classes = np.unique(train_data[:, 0])
classes

array([1., 2., 3.], dtype=float128)

# Class gravity centers

In [4]:
gravity_centers = []
for class_ in classes:
    # mask class data
    class_data = train_data[train_data[:, 0] == class_]
    #remve class column
    class_data = class_data[:, 1:]
    #mean by column
    class_gravity_centers = np.mean(class_data, axis=0)
    gravity_centers.append(class_gravity_centers)
gravity_centers = np.array(gravity_centers)
gravity_centers

array([[5.028, 3.48 , 1.46 , 0.248],
       [6.012, 2.776, 4.312, 1.344],
       [6.576, 2.928, 5.64 , 2.044]], dtype=float128)

In [5]:
mv = np.mean(train_data[:, 1:], axis=0)
mv

array([5.872     , 3.06133333, 3.804     , 1.212     ], dtype=float128)

In [7]:
sd = np.std(train_data[:, 1:], axis=0, ddof=1)  #excel defualt to one degree of freedom nympy to 0
sd

array([0.85685787, 0.46843423, 1.81498567, 0.76951371], dtype=float128)

In [8]:
gravity_centers_std = (gravity_centers - mv) / sd
gravity_centers_std

array([[-0.98499417,  0.89375762, -1.29147025, -1.25273921],
       [ 0.16338766, -0.60912144,  0.27989202,  0.1715369 ],
       [ 0.82160651, -0.28463619,  1.01157824,  1.08120231]],
      dtype=float128)

## Standardised training set

In [18]:
test_data_std = (test_data[:, 1:] - mv) / sd


In [41]:
def distance(a, b):
    return np.sum((a - b) ** 2)

In [56]:
distance_tab = []

for data_row in test_data_std:
    distances = []
    for center in gravity_centers_std:
        distances.append(distance(data_row, center))
    distance_tab.append(distances)
distance_tab = np.array(distance_tab)
distance_tab

array([[1.06089876e+00, 6.06641673e+00, 1.41036000e+01],
       [7.52011660e-02, 6.90701045e+00, 1.39171802e+01],
       [4.64934039e-02, 7.89737907e+00, 1.50152168e+01],
       [7.44440300e-02, 7.45682329e+00, 1.47938286e+01],
       [5.13660312e-01, 7.60663421e+00, 1.58276432e+01],
       [7.38710613e-01, 6.92197899e+00, 1.51278929e+01],
       [2.57150466e-01, 6.18993447e+00, 1.26661895e+01],
       [1.82957663e+00, 1.39006018e+01, 2.04236233e+01],
       [2.67089709e+00, 1.43824269e+01, 2.01502028e+01],
       [6.84758625e-01, 6.77313635e+00, 1.49060420e+01],
       [3.82768116e-01, 7.36422431e+00, 1.54468357e+01],
       [3.16920616e-01, 7.70998011e+00, 1.45281658e+01],
       [1.26023047e-01, 9.96603387e+00, 1.77232909e+01],
       [1.59880992e+00, 8.73205988e+00, 1.79329377e+01],
       [4.06036906e-02, 7.51787359e+00, 1.49278872e+01],
       [1.52284188e-02, 8.37833669e+00, 1.57282738e+01],
       [6.73755952e+00, 8.74096445e+00, 1.85215633e+01],
       [9.06108091e-01, 9.32267

# Results

In [50]:
mins = np.argmin(distance_tab, axis=1) + 1
mins

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 2, 3, 3,
       3, 2, 3, 3, 3, 3, 3, 3, 2])

In [55]:
correct_count = np.sum(mins == test_data[:, 0])
correct_count

67

In [54]:
incorrect_count = np.sum(mins != test_data[:, 0])
incorrect_count

8